# Item Collaborative Filtering
* See `ItemCollaborativeFilteringBase.ipynb` for algorithm details
* The weights here are the reciprocal of the l2-norm between the item embeddings

In [1]:
name = "ItemCF.Embed";

In [2]:
using NBInclude
@nbinclude("ItemCFBase.ipynb");

## Compute cosine correlations

In [3]:
function normalize_rows(X)
    σ = std(X, dims = 2)
    μ = mean(X, dims = 2)
    (X .- μ) ./ σ
end

function get_embed_matrix_outdir(power)
    # if the matrix is already stored on disk, return its filepath
    # otherwise, regenerate the matrix and store it to disk
    outdir = "$name/$(hash(power))"
    if ispath("../../data/alphas/$outdir")
        return outdir
    end

    @debug "generating similarity matrix for $power-norm"
    item_embeddings = normalize_rows(
        vcat(
            [collect(read_params("MatrixFactorization.$K")["A"]') for K in [10, 20, 40]]...,
        ),
    )
    training = get_residuals("training", residual_alphas)
    S = zeros(maximum(training.item), maximum(training.item))

    kernel = item_embeddings
    p = power
    @tprogress Threads.@threads for j = 1:size(S)[1]
        dists = map(x -> norm(x, p), eachslice(kernel .- kernel[:, j], dims = 2))
        S[:, j] = 1 ./ dists
    end

    # remove Inf
    clamp!(S, -1e9, 1e9)
    S = convert.(Float32, S)
    write_params(Dict("S" => S), outdir = outdir)
    outdir
end;

## Setup hyperparameters

In [4]:
item_cf_params = [
    cf_params(
        name = "ItemCF.Embed.$K",
        training_residuals = ["UserItemBiases"],
        validation_residuals = ["UserItemBiases"],
        neighborhood_type = "abs",
        S = get_embed_matrix_outdir(2),
        K = K,
        λ = [1.0, 1.0, 0.0],
    ) for K in [2^8]
];

[ Debug: 20220225 13:59:52 generating similarity matrix for 2-norm
Progress: 100%|███████████████████████████| Time: 0:00:09 ( 7.99 ms/it)


## Train models

In [5]:
for param in item_cf_params
    optimize_model(param)
end

Progress: 100%|███████████████████████████| Time: 0:01:13 (86.05 ms/it)
[ Debug: 20220225 14:02:53 loss: Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float32}}(1.5124373,-0.040910013,0.0021147113,-1.3616091) β: Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float32}}(0.6240272,0.02345366,-0.43045282,6.719317): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float32}, Float32, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float32}}(1.0,1.0,0.0,0.0), Dual{ForwardDiff.Tag{var"#validation

Iter     Function value   Gradient norm 
     0     1.512437e+00     1.361609e+00
 * Current step size: 1.0
 * time: 0.02502608299255371
 * g(x): Float32[-0.040910013, 0.0021147113, -1.3616091]
 * x: Float32[1.0, 1.0, 0.0]


Progress: 100%|███████████████████████████| Time: 0:01:50 ( 0.13  s/it)
[ Debug: 20220225 14:05:07 loss: Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float32}}(1.5006993,0.06975965,-0.008996509,0.025124907) β: Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float32}}(2.817305,2.9766443,-1.6654081,1.0544095): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float32}, Float32, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float32}}(1.04091,1.0,0.0,0.0), Dual{ForwardDiff.Tag{var"#validatio

     1     1.499969e+00     6.881835e-02
 * Current step size: 0.9804077
 * time: 267.7211310863495
 * g(x): Float32[0.06881835, -0.009043385, 0.025436264]
 * x: Float32[1.0401084, 0.9979267, 1.3349321]


Progress: 100%|███████████████████████████| Time: 0:01:54 ( 0.13  s/it)
[ Debug: 20220225 14:09:29 loss: Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float32}}(1.4953718,0.054921303,-0.008257365,0.023662424) β: Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float32}}(2.5673468,2.7637973,-1.539749,0.9719316): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float32}, Float32, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float32}}(0.97551835,1.0,0.0,0.0), Dual{ForwardDiff.Tag{var"#valid

...

Progress: 100%|███████████████████████████| Time: 0:01:54 ( 0.13  s/it)
[ Debug: 20220225 16:19:07 loss: Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float32}}(1.4473007,0.005500708,-0.00095743925,0.45962283) β: Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float32}}(0.92667246,0.3287584,-0.61629605,26.356073): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float32}, Float32, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float32}}(2.1193802,1.0,0.0,0.0), Dual{ForwardDiff.Tag{var"#va

    20     1.447361e+00     3.561541e-01
 * Current step size: 0.8391547
 * time: 8569.871547937393
 * g(x): Float32[0.0035618471, -0.0010011452, 0.35615408]
 * x: Float32[2.086667, 1.0360268, 0.0067093987]


Progress: 100%|███████████████████████████| Time: 0:01:55 ( 0.13  s/it)
[ Debug: 20220225 16:27:53 loss: Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float32}}(1.7427762,-0.023969194,1.2127275e-8,0.48515943) β: Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float32}}(3.9041393e-7,1.5410148,-1.0812325e-6,-31.25923): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float32}, Float32, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#23"{var"#20#22"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float32}}, RatingsDataset, RatingsDataset}, Float32}}(3.4962547,1.0,0.0,0.0), Dual{ForwardDiff.Tag{var"

    21     1.446803e+00     1.595697e-01
 * Current step size: 0.06576269
 * time: 9598.028109073639
 * g(x): Float32[-0.005660559, -0.0005341878, -0.15956967]
 * x: Float32[2.1793654, 1.0361378, 0.0039308807]


Progress:   6%|█▋                         |  ETA: 0:00:53 (65.55 ms/it)IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

